In [1]:
import streaminghub_datamux as dm

In [2]:
class LogWriter(dm.SinkTask):

    def __init__(self) -> None:
        super().__init__()

    def step(self, input) -> int | None:
        print(input)

In [3]:
# constants
dataset = "ADHD_SIN"
timeout = 10
screen_wh = (1920, 1080)
diag_dist = (21, 22)
freq = 60

# hyperparameters
vt = 10

In [4]:
mode = "remote"

# setup datamux api
if mode == "local":
    api = dm.API()

elif mode == "remote":
    api = dm.RemoteAPI("websocket", "msgpack")
    api.connect("full-darling-gar.ngrok-free.app", 80)

else:
    raise ValueError(mode)

Loaded streaminghub_datamux.rpc.client:websocket
Loaded streaminghub_datamux.rpc.codec:msgpack


In [5]:
streams = api.list_collection_streams(dataset)  # for recorded data (ADHD_SIN)
# streams = api.list_live_streams("pupil_core")  # for live data (pupil_core)

# get the first stream
eye_stream = streams[0]
print(eye_stream)

name='eye movements' description='' unit='pixel, mm' frequency=60 fields={'lx': Field(name='left x', description='', dtype=<class 'numpy.float32'>), 'ly': Field(name='left y', description='', dtype=<class 'numpy.float32'>), 'ld': Field(name='left d', description='', dtype=<class 'numpy.float32'>), 'rx': Field(name='right x', description='', dtype=<class 'numpy.float32'>), 'ry': Field(name='right y', description='', dtype=<class 'numpy.float32'>), 'rd': Field(name='right d', description='', dtype=<class 'numpy.float32'>)} index={'t': Field(name='timestamp', description='', dtype=<class 'numpy.float32'>)} node=Node(id='ADHD_SIN', device=None, uri=None, inputs={}, outputs={}) attrs={'subject': '034', 'noise': '0', 'task': '17', 'collection': 'ADHD_SIN', 'id': 'eye', 'mode': 'replay'}


In [6]:
# define a transform to map data into (t,x,y,d) format and handle missing values
merge = dm.ExpressionMap(
    {
        "t": "t",
        "x": "(lx + rx) / 2",
        "y": "(ly + ry) / 2",
        "d": "(ld + rd) / 2",
    }
)

cast = dm.ExpressionMap(
    {
        "t": "float(t)",
        "x": "float(x)",
        "y": "float(y)",
        "d": "float(d)",
    }
)

In [7]:
# define pipeline
pipeline_A = dm.Pipeline(
    api.attach(eye_stream, transform=merge).with_name("eye"),
    dm.Filter("not (isnan(x) or isnan(y) or isnan(d))"),
    dm.Transform(cast),
    LogWriter().with_name("log"),
).with_name("simple_logger")
# run pipeline
pipeline_A.run(timeout)

{'t': 0.25099999999997635, 'x': 1072.0, 'y': 726.0, 'd': 4.595}
{'t': 0.26900000000000546, 'x': 1060.5, 'y': 769.0, 'd': 4.625}
{'t': 0.28700000000003456, 'x': 1053.5, 'y': 773.5, 'd': 4.645}
{'t': 0.3049999999998363, 'x': 1075.0, 'y': 722.5, 'd': 4.6899999999999995}
{'t': 0.3229999999998654, 'x': 1043.5, 'y': 759.5, 'd': 4.685}
{'t': 0.3409999999998945, 'x': 1056.5, 'y': 786.5, 'd': 4.67}
{'t': 0.35799999999994725, 'x': 1065.0, 'y': 782.0, 'd': 4.63}
{'t': 0.3769999999999527, 'x': 1049.5, 'y': 776.0, 'd': 4.575}
{'t': 0.3949999999999818, 'x': 1068.0, 'y': 776.0, 'd': 4.585}
{'t': 0.41200000000003456, 'x': 1048.5, 'y': 779.0, 'd': 4.575}
{'t': 0.42899999999985994, 'x': 1058.0, 'y': 773.0, 'd': 4.605}
{'t': 0.4459999999999127, 'x': 1054.0, 'y': 778.0, 'd': 4.62}
{'t': 0.4619999999999891, 'x': 1059.5, 'y': 769.0, 'd': 4.609999999999999}
{'t': 0.47900000000004184, 'x': 1057.5, 'y': 765.5, 'd': 4.595000000000001}
{'t': 0.4959999999998672, 'x': 1056.5, 'y': 770.0, 'd': 4.57}
{'t': 0.5119999